Import della libreria transformer

In [4]:
from transformers import pipeline

c:\Users\pietr\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import torch

In [3]:
torch.cuda.is_available()

True

In [3]:
Bart = pipeline(model = "facebook/bart-large-mnli", device=0)

In [5]:
distilBert = pipeline(model = "typeform/distilbert-base-uncased-mnli", device=0)

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Creazione della pipe per roberta (joeddav/xlm-roberta-large-xnli)

In [3]:
import sentencepiece

In [4]:
Roberta = pipeline(model = "joeddav/xlm-roberta-large-xnli", device=0)

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
rumor1 = "Naturalization decree in preparation: Lebanese passports for sale?!"
rumor2 = "Urgent  Ramallah Ministry of Health spokesman Kamal AlShakhra: We received 2000 doses of the American Moderna Corona vaccine and this batch will be designated for President Abbas the Fatah Central Committee and VIPs"
rumor3 = "Some reporters says that the Pope is dead because of some natural reason"

evidence1 = "The Information Office of the Presidency of the Republic: What was published by the French newspaper “Liberation” about the “selling” of Lebanese passports to non-Lebanese is false and baseless news."
evidence2 = "The Information Office of the Presidency of the Republic denies a false news broadcast by the MTV station about Baabda Palace preparing a decree naturalizing 4,000 people, and recalls that it had denied yesterday the false information published by the French magazine ‘Liberation’ about the same fabricated news."
evidence3 = "President Aoun received the Minister of Foreign Affairs and Expatriates, Dr. Abdullah Bouhabib, and the Minister of Social Affairs, Hector Hajjar, and discussed with them developments related to the file of displaced Syrians in Lebanon"
evidence4 = "Pope Francis visits Lebanon and Siria"
evidence5 = "President Aoun received the new British Ambassador to Lebanon, Hamish Cowell, and discussed with him Lebanese-British relations and ways to enhance them in all fields"
evidence6 = "Vaccination will begin in the middle of this month and we will provide the Gaza Strip with a share of all the vaccines we receive We expect the first batch of vaccines that the government has contracted to purchase to arrive at the end of this month of February or the beginning of the following month The Ministry of Health was directed to prepare a health protocol for the electoral process"
evidence7 = "SNL says that the current reports from unverified news outlets that the Pope is dead are true"

labels = ["AGREE", "DISAGREE", "OFF TOPIC"]
labels2 = ["SUPPORTS", "REFUTES", "NOT ENOUGH INFO"] #Quelle di partenza, bart tende al rifiuto
labelsRoberta = ["VALIDATES", "DISAGREE", "NOT ENOUGH INFO"] #Dovrebbe distinguere bene
labelsBart = ["CONFIRMS", "OPPOSE", "NOT ENOUGH INFO"] #Tende a confermare
#labels3 = ["AGREE", "DISAGREE"]





In [28]:
test_prompts = "("+rumor1+")"+" and "+"("+evidence4+")"

In [29]:
Bart(test_prompts, candidate_labels=labelsBart)

{'sequence': '(Naturalization decree in preparation: Lebanese passports for sale?!) and (Pope Francis visits Lebanon and Siria)',
 'labels': ['CONFIRMS', 'OPPOSE', 'NOT ENOUGH INFO'],
 'scores': [0.7658191919326782, 0.18122899532318115, 0.052951786667108536]}

In [10]:
distilBert(test_prompts, candidate_labels=labels2)

{'sequence': '(Some reporters says that the Pope is dead because of some natural reason) and (President Aoun received the new British Ambassador to Lebanon, Hamish Cowell, and discussed with him Lebanese-British relations and ways to enhance them in all fields)',
 'labels': ['REFUTES', 'SUPPORTS', 'NOT ENOUGH INFO'],
 'scores': [0.3851291537284851, 0.31933730840682983, 0.29553350806236267]}

In [43]:
Roberta(test_prompts, candidate_labels=labels3)

{'sequence': '(Naturalization decree in preparation: Lebanese passports for sale?!) and (Pope Francis visits Lebanon and Siria)',
 'labels': ['VALIDATES', 'DISAGREE', 'NOT ENOUGH INFO'],
 'scores': [0.7232279181480408, 0.23452650010585785, 0.042245663702487946]}

Carico il test dal drive

In [7]:
import json

with open('English_dev.json', 'r') as f:
    devSet = json.load(f)

Prendo solo il primo esempio e vedo come classificare

In [7]:
def validateResult(result):
  if result["scores"][0] > 0.9:
    if result["labels"][0]=="AGREE": return "SUPPORTS"
    if result["labels"][0]=="DISAGREE": return "REFUTES"
    return "NOT ENOUGH INFO"
  else:
    return "NOT ENOUGH INFO"

In [16]:
len(testSet[9]['timeline'])

564

Per ogni rumor vado a memorizzare tutte le authority con la loro probabilità e le label prodotta

In [ ]:
risultati = {}
i = 1
#Per ogni elemento del training set
for rumor in testSet:
  #rumor = trainSet[0]
  print("Elaborando il rumor: "+str(i)+"/"+str(len(testSet)))
  i = i + 1
  risultati[rumor["id"]] = []
  #Per ogni elemento della timeline vado a calcolarne la classe
  for element in rumor["timeline"]:
    rumorString = rumor["rumor"]
    evidence = element[2]
    prompts = "("+rumorString+")"+" and "+"("+evidence+")"
    #ris = validateResult(Bart(prompts, candidate_labels=labels2))
    ris = Bart(prompts, candidate_labels=labels2)

    #Aggiungo al dizionario l'elaborazione attuale
    element.append(ris["labels"])
    element.append(ris["scores"])
    risultati[rumor["id"]].append(element)

    #Se il risultato è "SUPPRTS" o "REFUTES"
    #if ris in ["REFUTES","SUPPORTS"]:
      #break



In [ ]:
risultati

LABELING
In quesata versione la label più probabile da la label al rumor

In [12]:
listaDizionari = []
#Per ogni elemento del dizionario e quindi per ogni rumor prendo la label più probabile
for chiave in risultati:
    lista = risultati[chiave]
    
    #Ordino le authority in base alla percentuale
    lista.sort(key=lambda x: x[4][0], reverse=True)

    #Recupero la label più probabile e recupero i primi 5 tweet più probabili con quella label
    etichetta = lista[0][3][0]
    listaAuthority = []

    for authority in lista:
        if len(listaAuthority) == 5:
            break
        if authority[3][0] == etichetta:
            listaAuthority.append(authority)
    
    listaAuthority2 = []
    for elemento in listaAuthority:
        listaAuthority2.append([elemento[0],elemento[1],elemento[2],elemento[4][0]])

    #Creazione dell'elemento json
    dizionario = {
        "id":chiave,
        "predicted_label":etichetta,
        "predicted_evidence":listaAuthority2
    }
    listaDizionari.append(dizionario)

Salvo il file come risultati del modello

In [15]:
with open("Facebook-Bart_Dev_results.json", "w") as fr:
    json.dump(listaDizionari, fr)

In [ ]:
listaDizionari

Creo le funzioni per automatizzare quello di sopra

In [8]:
def calcolaProbabilita(modello, dati):
    risultati = {}
    i = 1
    #Per ogni elemento del training set
    for rumor in dati:
        #rumor = trainSet[0]
        print("Elaborando il rumor: "+str(i)+"/"+str(len(dati)))
        i = i + 1
        risultati[rumor["id"]] = []
        #Per ogni elemento della timeline vado a calcolarne la classe
        for element in rumor["timeline"]:
            rumorString = rumor["rumor"]
            evidence = element[2]
            prompts = "("+rumorString+")"+" and "+"("+evidence+")"
            #ris = validateResult(Bart(prompts, candidate_labels=labels2))
            ris = modello(prompts, candidate_labels=labelsBart)

            #Aggiungo al dizionario l'elaborazione attuale
            element.append(ris["labels"])
            element.append(ris["scores"])
            risultati[rumor["id"]].append(element)
            
    return risultati

In [8]:
#labelsBart = ["CONFIRMS", "OPPOSE", "NOT ENOUGH INFO"]
#labelsRoberta = ["VALIDATES", "DISAGREE", "NOT ENOUGH INFO"]
#labels = ["AGREE", "DISAGREE", "OFF TOPIC"]
#labels2 = ["SUPPORTS", "REFUTES", "NOT ENOUGH INFO"]
def convertiLabels(labelsUsate):
    risLabels = []
    for app in labelsUsate:
        if app == "AGREE": risLabels.append("SUPPORTS")
        if app == "DISAGREE": risLabels.append("REFUTES")
        if app == "OFF TOPIC": risLabels.append("NOT ENOUGH INFO")

    return risLabels

In [9]:
def calcolaProbabilitaDifferentLabels(modello, dati):
    risultati = {}
    i = 1
    #Per ogni elemento del training set
    for rumor in dati:
        #rumor = trainSet[0]
        print("Elaborando il rumor: "+str(i)+"/"+str(len(dati)))
        i = i + 1
        risultati[rumor["id"]] = []
        #Per ogni elemento della timeline vado a calcolarne la classe
        for element in rumor["timeline"]:
            rumorString = rumor["rumor"]
            evidence = element[2]
            prompts = "("+rumorString+")"+" and "+"("+evidence+")"
            #ris = validateResult(Bart(prompts, candidate_labels=labels2))
            ris = modello(prompts, candidate_labels=labels)

            #Aggiungo al dizionario l'elaborazione attuale
            element.append(convertiLabels(ris["labels"]))
            element.append(ris["scores"])
            risultati[rumor["id"]].append(element)
            
    return risultati

In [10]:
def calcolaJson(probabilita):
    listaDizionari = []
    #Per ogni elemento del dizionario e quindi per ogni rumor prendo la label più probabile
    for chiave in probabilita:
        lista = probabilita[chiave]
        
        #Ordino le authority in base alla percentuale
        lista.sort(key=lambda x: x[4][0], reverse=True)

        #Recupero la label più probabile e recupero i primi 5 tweet più probabili con quella label
        etichetta = lista[0][3][0]
        listaAuthority = []

        for authority in lista:
            if len(listaAuthority) == 5:
                break
            if authority[3][0] == etichetta:
                listaAuthority.append(authority)
        
        listaAuthority2 = []
        for elemento in listaAuthority:
            listaAuthority2.append([elemento[0],elemento[1],elemento[2],elemento[4][0]])

        #Creazione dell'elemento json
        dizionario = {
            "id":chiave,
            "predicted_label":etichetta,
            "predicted_evidence":listaAuthority2
        }
        listaDizionari.append(dizionario)
    
    return listaDizionari

1) Calcolo le probabilità con distilBert
2) Creo il json
3) Salvo tutto

In [ ]:
#probabilita = calcolaProbabilita(Bart,devSet)
probabilita = calcolaProbabilitaDifferentLabels(Bart,devSet)

file = calcolaJson(probabilita)
  

In [14]:
with open("Results/Verification/Bart_verification_results_v3.json", "w") as fr:
    json.dump(file, fr)

#Scrivo il file di testo per le la verifica delle evidence
with open("Results/Evidence/Bart_evidence_results_v3.txt", "w") as fe:
    for rumor in file:
        #Scrivo le cose che servono per la verifica delle evidence
        i = 1
        for evid in rumor['predicted_evidence']:
            fe.write(rumor['id']+"\tQ0\t"+evid[1]+"\t"+str(i)+"\t"+str(evid[3])+"\tIO\n")
            i = i+1

In [12]:
#probabilita = calcolaProbabilita(distilBert,devSet)
probabilita = calcolaProbabilitaDifferentLabels(distilBert,devSet)

file = calcolaJson(probabilita)



Elaborando il rumor: 1/32


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Elaborando il rumor: 2/32
Elaborando il rumor: 3/32
Elaborando il rumor: 4/32
Elaborando il rumor: 5/32
Elaborando il rumor: 6/32
Elaborando il rumor: 7/32
Elaborando il rumor: 8/32
Elaborando il rumor: 9/32
Elaborando il rumor: 10/32
Elaborando il rumor: 11/32
Elaborando il rumor: 12/32
Elaborando il rumor: 13/32
Elaborando il rumor: 14/32
Elaborando il rumor: 15/32
Elaborando il rumor: 16/32
Elaborando il rumor: 17/32
Elaborando il rumor: 18/32
Elaborando il rumor: 19/32
Elaborando il rumor: 20/32
Elaborando il rumor: 21/32
Elaborando il rumor: 22/32
Elaborando il rumor: 23/32
Elaborando il rumor: 24/32
Elaborando il rumor: 25/32
Elaborando il rumor: 26/32
Elaborando il rumor: 27/32
Elaborando il rumor: 28/32
Elaborando il rumor: 29/32
Elaborando il rumor: 30/32
Elaborando il rumor: 31/32
Elaborando il rumor: 32/32


In [15]:
with open("Results/Verification/distilBert_verification_results_v3.json", "w") as fr:
    json.dump(file, fr)

#Scrivo il file di testo per le la verifica delle evidence
with open("Results/Evidence/distilBert_evidence_results_v3.txt", "w") as fe:
    for rumor in file:
        #Scrivo le cose che servono per la verifica delle evidence
        i = 1
        for evid in rumor['predicted_evidence']:
            fe.write(rumor['id']+"\tQ0\t"+evid[1]+"\t"+str(i)+"\t"+str(evid[3])+"\tIO\n")
            i = i+1

: 

Stessa cosa per Roberta

In [11]:
#probabilita = calcolaProbabilita(Roberta,devSet)
probabilita = calcolaProbabilitaDifferentLabels(Roberta,devSet)

file = calcolaJson(probabilita)



Elaborando il rumor: 1/32


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Elaborando il rumor: 2/32
Elaborando il rumor: 3/32
Elaborando il rumor: 4/32
Elaborando il rumor: 5/32
Elaborando il rumor: 6/32
Elaborando il rumor: 7/32
Elaborando il rumor: 8/32
Elaborando il rumor: 9/32
Elaborando il rumor: 10/32
Elaborando il rumor: 11/32
Elaborando il rumor: 12/32
Elaborando il rumor: 13/32
Elaborando il rumor: 14/32
Elaborando il rumor: 15/32
Elaborando il rumor: 16/32
Elaborando il rumor: 17/32
Elaborando il rumor: 18/32
Elaborando il rumor: 19/32
Elaborando il rumor: 20/32
Elaborando il rumor: 21/32
Elaborando il rumor: 22/32
Elaborando il rumor: 23/32
Elaborando il rumor: 24/32
Elaborando il rumor: 25/32
Elaborando il rumor: 26/32
Elaborando il rumor: 27/32
Elaborando il rumor: 28/32
Elaborando il rumor: 29/32
Elaborando il rumor: 30/32
Elaborando il rumor: 31/32
Elaborando il rumor: 32/32


In [14]:
with open("Results/Verification/Roberta_verification_results_v3.json", "w") as fr:
    json.dump(file, fr)

#Scrivo il file di testo per le la verifica delle evidence
with open("Results/Evidence/Roberta_evidence_results_v3.txt", "w") as fe:
    for rumor in file:
        #Scrivo le cose che servono per la verifica delle evidence
        i = 1
        for evid in rumor['predicted_evidence']:
            fe.write(rumor['id']+"\tQ0\t"+evid[1]+"\t"+str(i)+"\t"+str(evid[3])+"\tIO\n")
            i = i+1
        